In [61]:
import pyeuropeana.apis as apis
import pyeuropeana.utils as utils
import os
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [62]:
#setting enviroment variable
os.environ['EUROPEANA_API_KEY'] = 'api2demo'

In [198]:
 def search_api(qf, n_rows):
    response = apis.search(
    query = '*:*',
    qf=qf,
    rows = n_rows, 
    profile='rich'
    )
    df_search=utils.search2df(response).europeana_id
    return df_search

In [199]:
def record_api(items_id):
    df_list=[]
    for item in items_id:
        data=apis.record(f'{item}')
        data_jnorm=pd.json_normalize(data)
        df_list.append(data_jnorm)
    df_jnorm_tot = pd.concat(df_list, ignore_index=True, axis=0)
    return df_jnorm_tot

In [465]:
def record_api_proxy(items_id):
    df_list=[]
    for item in items_id:
        data=apis.record(f'{item}')
        df_0=pd.json_normalize(data,['object','proxies'])
        df_proxy_provider=df_0.iloc[2] #selcting provider proxy - there are the info I am interested in
        df_proxy_provider=pd.DataFrame(df_proxy_provider)
        df_proxy_provider=df_proxy_provider.transpose()
        df_proxy_provider['item_id']=item
        df_list.append(df_proxy_provider)
    df_proxy_tot = pd.concat(df_list, ignore_index=True, axis=0)
    return df_proxy_tot

In [466]:
def retrieve_norm_data(query, n_rows):
    search_results_list=search_api(query, n_rows)
    record_data=record_api_proxy(search_results_list)
    return record_data

In [467]:
def fin_records(query):
    query=query
    df=retrieve_norm_data(query,5)
    list_of_item_ids_web=[]
    for item_id in list(df.item_id):
        item_id=f'https://search-api.europeana.eu/record{item_id}.json?wskey=api2demo'
        list_of_item_ids_web.append(item_id)
    return df, list_of_item_ids_web            

In [468]:
query='(edm_datasetName:2048218*)'

In [469]:
 df, lst=fin_records(query)

In [470]:
df

,about,proxyIn,proxyFor,lineage,edmType,europeanaProxy,dcIdentifier.def,dcLanguage.def,dcType.def,dctermsSpatial.def,year.def,dcTitle.en,dcContributor.fr,dcCreator.fr,dcDate.fr,dcLanguage.fr,dcPublisher.fr,dcSubject.def,dcSubject.en,dcTitle.fr,dcType.en,dctermsCreated.fr,dctermsIsPartOf.en,item_id
0,/proxy/provider/2048218/europeana_fashion_mad_638,[/aggregation/provider/2048218/europeana_fashi...,/item/2048218/europeana_fashion_mad_638,NaN,TEXT,False,[mad 638],NaN,[http://thesaurus.europeanafashion.eu/thesauru...,NaN,NaN,NaN,"[Jean-Paul Leclercq (Author), Lydia Kamitsis (...","[Jean-Paul Leclercq, Lydia Kamitsis, Pierre Ar...",[1997],[français],"[Réunion des Musées nationaux, Musée de la Mod...",[http://thesaurus.europeanafashion.eu/thesauru...,[Object Type: fashion exhibition],[L'album du Musée de la Mode et du Textile.],"[Object Type: book, Object Type: exhibition ca...",[1997],[Europeana XX: Century of Change],/2048218/europeana_fashion_mad_638
1,/proxy/provider/2048218/europeana_fashion_mad_609,[/aggregation/provider/2048218/europeana_fashi...,/item/2048218/europeana_fashion_mad_609,NaN,TEXT,False,[mad 609],NaN,[http://thesaurus.europeanafashion.eu/thesauru...,NaN,NaN,NaN,"[Merry Foresta (Author), Willis Hartshorn (Aut...","[Merry Foresta, Willis Hartshorn, Katell Le Bo...",[1992],[français],[Musée de la Mode et du Textile des Arts Décor...,[http://thesaurus.europeanafashion.eu/thesauru...,[Object Type: fashion photograph],[Man Ray. Les années Bazaar. Photographies de ...,"[Object Type: book, Object Type: exhibition ca...",[1992],[Europeana XX: Century of Change],/2048218/europeana_fashion_mad_609
2,/proxy/provider/2048218/europeana_fashion_mad_573,[/aggregation/provider/2048218/europeana_fashi...,/item/2048218/europeana_fashion_mad_573,NaN,TEXT,False,[mad 573],NaN,[http://thesaurus.europeanafashion.eu/thesauru...,NaN,NaN,NaN,"[François Baudot (Author), Marie-Josée Lepicar...","[François Baudot, Marie-Josée Lepicard, Jacque...",[1989],[français],[Musée des Arts Décoratifs de Paris],[http://thesaurus.europeanafashion.eu/thesauru...,[Object Type: fabric],[Nos années 80. 29 novembre 1989-1er avril 1990],"[Object Type: book, Object Type: exhibition ca...",[1989],[Europeana XX: Century of Change],/2048218/europeana_fashion_mad_573
3,/proxy/provider/2048218/europeana_fashion_mad_557,[/aggregation/provider/2048218/europeana_fashi...,/item/2048218/europeana_fashion_mad_557,NaN,TEXT,False,[mad 557],NaN,[http://thesaurus.europeanafashion.eu/thesauru...,NaN,NaN,NaN,[Union des Arts Décoratifs (Author)],[Union des Arts Décoratifs],[1988],[français],[Union Centrale des Arts Décoratifs],[http://thesaurus.europeanafashion.eu/thesauru...,[Object Type: fashion exhibition],"[Histoires de mode d'hier et d'aujourd'hui, do...","[Object Type: book, Object Type: exhibition ca...",[1988],[Europeana XX: Century of Change],/2048218/europeana_fashion_mad_557
4,/proxy/provider/2048218/europeana_fashion_mad_511,[/aggregation/provider/2048218/europeana_fashi...,/item/2048218/europeana_fashion_mad_511,NaN,TEXT,False,[mad 511],NaN,[http://thesaurus.europeanafashion.eu/thesauru...,NaN,NaN,NaN,"[Martand Singh (Author), Asha Sarabhai (Author...","[Martand Singh, Asha Sarabhai, François Mathey]",[1985],[français],[Musée des Arts Décoratifs de Paris],[http://thesaurus.europeanafashion.eu/thesauru...,[Object Type: fabric],"[Les textiles de l'Inde : l'or, la laine et la...","[Object Type: book, Object Type: exhibition ca...",[1985],[Europeana XX: Century of Change],/2048218/europeana_fashion_mad_511


In [471]:
lst

['https://search-api.europeana.eu/record/2048218/europeana_fashion_mad_638.json?wskey=api2demo',
 'https://search-api.europeana.eu/record/2048218/europeana_fashion_mad_609.json?wskey=api2demo',
 'https://search-api.europeana.eu/record/2048218/europeana_fashion_mad_573.json?wskey=api2demo',
 'https://search-api.europeana.eu/record/2048218/europeana_fashion_mad_557.json?wskey=api2demo',
 'https://search-api.europeana.eu/record/2048218/europeana_fashion_mad_511.json?wskey=api2demo']

In [285]:
EuSCREEN
# query='(edm_datasetName:2051914* AND proxy_dc_terms_temporal:*)' gives no result suggesting there are no items with dc_terms_temporal in the dataset, same ofr dc_terms_spatial in contrast with notes aggregator
# checked in both italian and english

# dc description in both en en nl, dc subject in both nl and en, dc title in both nl and en, dc terms alternative in both, dc Spatial in both en and nl
data_1='https://search-api.europeana.eu/record/2051906/data_euscreenXL_https___www_openbeelden_nl_media_9972.json?wskey=api2demo'
#dc format dc type in english
data_2='https://search-api.europeana.eu/record/2051918/data_euscreenXL_EUS_FFDCE2AB00B2936EC29312E656622105.json?wskey=api2demo',
#dc type and dc format in english only, dc terms alternative in both en and it
data_3='https://search-api.europeana.eu/record/2051933/data_euscreenXL_EUS_FFE4654D917AF1BD85344825D9B62172.json?wskey=api2demo'
#dctype only in french
data_4 ='https://search-api.europeana.eu/record/2051908/data_euscreenXL_ina_VDX14007632.json?wskey=api2demo',

MUSEU
#dc_termsmedium abd dc format in french only
data_5= 'https://search-api.europeana.eu/record/2048001/AP_10450606.json?wskey=api2demo',

#dc_termsmedium  dc format in dutch only
data_6='https://search-api.europeana.eu/record/2048001/AP_10407814.json?wskey=api2demo''

#dc description and title in italian
data_7='https://search-api.europeana.eu/record/226/96_eikonprojekt_RM_2010.json?wskey=api2demo',

#dcterms medium and type in french only
data_8='https://search-api.europeana.eu/record/2048001/AP_10450615.json?wskey=api2demo'

#dctype and dcspatial in french and english, format in eng nl, and fr
data_9='https://search-api.europeana.eu/record/322/Museu_ProvidedCHO_Mus_es_Royaux_d_Art_et_d_Histoire_94355.json?wskey=api2demo',

Fashion

#dctempral in dutch
data_10='https://search-api.europeana.eu/record/2048230/europeana_fashion_9927.json?wskey=api2demo'


#dcterms spatial in dutch
data_11'https://search-api.europeana.eu/record/2048230/europeana_fashion_920.json?wskey=api2demo',

# dctitle in french
data_12='https://search-api.europeana.eu/record/2048218/europeana_fashion_mad_638.json?wskey=api2demo'





In [455]:
 def search_api_alt(qf, n_rows):
    response = apis.search(
    query = '*:*',
    qf=qf,
    rows = n_rows, 
    profile='rich'
    )
    df_search=utils.search2df(response)
    return df_search

In [456]:
search_api_alt(query, 2)